In [1]:
from huggingface_hub import login
login("TU_TOKEN_AQUI")  


In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True
)
model.eval()


/Users/nathcab/interpretabilidad-transfromers/new_env/lib/python3.13/site-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/nathcab/interpretabilidad-transfromers/new_env/lib/python3.13/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some parameters are on the meta device because they were offloaded to the disk.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [17]:
# Hook: capturar la salida del módulo MLP en la capa 8
mlp_outputs = []

def capture_mlp_output(module, input, output):
    mlp_outputs.append(output.detach().cpu())

# Conectamos el hook a la capa 8 del modelo
layer_idx = 8
mlp_module = model.model.layers[layer_idx].mlp
hook_handle = mlp_module.register_forward_hook(capture_mlp_output)


In [12]:
from datasets import load_dataset

# Activar confianza en código remoto para oscar
dataset = load_dataset(
    "oscar",
    "unshuffled_deduplicated_es",
    split="train[:1%]",
    trust_remote_code=True
)

# Limpiar textos vacíos
texts = [item["text"] for item in dataset if item["text"].strip() != ""]

print(f"Total de textos cargados: {len(texts)}")


Generating train split:   0%|          | 0/56326016 [00:00<?, ? examples/s]

Total de textos cargados: 563260


In [18]:
import json

# Guardar como archivo .jsonl (una línea por texto)
with open("textos_oscar_1porciento.jsonl", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(json.dumps({"text": text}) + "\n")

print(" Archivo 'textos_oscar_1porciento.jsonl' guardado correctamente.")


 Archivo 'textos_oscar_1porciento.jsonl' guardado correctamente.


In [19]:
import json

texts = []
with open("textos_oscar_1porciento.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        texts.append(json.loads(line)["text"])

print(f"Textos cargados desde disco: {len(texts)}")


Textos cargados desde disco: 563260


In [ ]:
# bloque de prueba 
from torch.utils.data import DataLoader
import torch

batch_size = 4
max_length = 256

def collate_fn(batch_texts):
    return tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_fn)

# Asegurarnos de limpiar la lista de activaciones antes
mlp_outputs.clear()

# Forward por batches y captura
with torch.no_grad():
    for i, batch in enumerate(loader):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        _ = model(**batch)
        print(f"Lote {i+1}: capturados {len(mlp_outputs)} tensores")
        if i == 9:  # ⚠️ SOLO PARA PRUEBA: hacer solo 10 lotes
            break


Lote 1: capturados 1 tensores
Lote 2: capturados 2 tensores
Lote 3: capturados 3 tensores
Lote 4: capturados 4 tensores
Lote 5: capturados 5 tensores
Lote 6: capturados 6 tensores
Lote 7: capturados 7 tensores
Lote 8: capturados 8 tensores
Lote 9: capturados 9 tensores
Lote 10: capturados 10 tensores


In [22]:
# Bloque de prueba 
import os
import numpy as np

output_dir = "activaciones_mlp8"
os.makedirs(output_dir, exist_ok=True)

for i, tensor in enumerate(mlp_outputs):
    np.save(os.path.join(output_dir, f"lote_{i:04d}.npy"), tensor.numpy())

print(f" Guardados {len(mlp_outputs)} tensores en '{output_dir}/'")


 Guardados 10 tensores en 'activaciones_mlp8/'


In [24]:
from torch.utils.data import DataLoader
import torch
import os
import numpy as np

# Parámetros
batch_size = 4
max_length = 256
output_dir = "activaciones_mlp8"
os.makedirs(output_dir, exist_ok=True)

# Asegurar pad_token
tokenizer.pad_token = tokenizer.eos_token

# Reconfigurar el hook por si reiniciaste
mlp_outputs.clear()
hook_handle.remove()  # Por si ya estaba registrado
hook_handle = mlp_module.register_forward_hook(capture_mlp_output)

# Tokenización por lote
def collate_fn(batch_texts):
    return tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length)

loader = DataLoader(texts, batch_size=batch_size, collate_fn=collate_fn)

# Procesamiento completo con guardado inmediato por lote
with torch.no_grad():
    for i, batch in enumerate(loader):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        
        # Limpia salida anterior
        mlp_outputs.clear()

        # Forward
        _ = model(**batch)

        # Guardar inmediatamente
        for j, tensor in enumerate(mlp_outputs):
            np.save(os.path.join(output_dir, f"lote_{i*batch_size + j:06d}.npy"), tensor.numpy())

        if (i + 1) % 10 == 0:
            print(f" Procesados y guardados {((i+1)*batch_size):,} textos")


 Procesados y guardados 40 textos
 Procesados y guardados 80 textos
 Procesados y guardados 120 textos
 Procesados y guardados 160 textos
 Procesados y guardados 200 textos
 Procesados y guardados 240 textos
 Procesados y guardados 280 textos
 Procesados y guardados 320 textos
 Procesados y guardados 360 textos
 Procesados y guardados 400 textos
 Procesados y guardados 440 textos
 Procesados y guardados 480 textos
 Procesados y guardados 520 textos
 Procesados y guardados 560 textos
 Procesados y guardados 600 textos
 Procesados y guardados 640 textos
 Procesados y guardados 680 textos
 Procesados y guardados 720 textos
 Procesados y guardados 760 textos
 Procesados y guardados 800 textos
 Procesados y guardados 840 textos
 Procesados y guardados 880 textos
 Procesados y guardados 920 textos
 Procesados y guardados 960 textos
 Procesados y guardados 1,000 textos
 Procesados y guardados 1,040 textos
 Procesados y guardados 1,080 textos
 Procesados y guardados 1,120 textos
 Procesados y 

KeyboardInterrupt: 